<a href="https://colab.research.google.com/github/Nubiancodingdelight/ACS-Project-Repository-/blob/main/Matrix_Creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#mounts google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install pysam


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 29.0 MB/s eta 0:00:00


In [3]:
# sets correct path
%cd drive/MyDrive/'ACS Research'/'Beginning Data'

/content/drive/MyDrive/ACS Research/Beginning Data


In [8]:
#imports
import pysam
import re
import tensorflow as tf
import numpy as np
#from pyspark.sql import SparkSession

In [13]:
#parse each row
def matrix_row(CIG, SE, PO):
  matrix_list= []
  seq_list= list(SE)
  cigar_list = re.split(r'(\d+)', CIG)[1::]
  if len(cigar_list) ==2:
    #if it is a matched a sequence of 1's based on the base are added
    if cigar_list[1] == 'M':
      for item in seq_list:
        bases = ['A','T','C','G']
        num_list = [0,0,0,0, PO]
        #if statement causes all values not in the base list to be ignored.
        num_list[bases.index(item)] = 1
        matrix_list.append(num_list)
    else:
      for time in range(int(len(cigar_list)/2)):
        if cigar_list[(2*time)+1] == 'M':
          for item in seq_list:
            bases = ['A','T','C','G']
            num_list = [0,0,0,0, PO]
            #if statement causes all values not in the base list to be ignored.
            num_list[bases.index(item)] = 1
            matrix_list.append(num_list)
  return matrix_list


In [29]:
#create matrix
samfile = pysam.AlignmentFile("bowtie2/SRR030257.bam", "rb")
final_matrix = []
for line in samfile:
  line_list = str(line).split("\t")
  #print(line_list)
  CIGAR = line_list[5]
  #print(CIGAR)
  SEQ = line_list[9]
  #print(SEQ)
  POS = line_list[3]
  #print(POS)
  #print(sum(matrix_row(CIGAR, SEQ, POS) , []))
  final_matrix.append(matrix_row(CIGAR, SEQ, POS))
  final_matrix =sum(final_matrix, [])
  print(final_matrix)

  #break

[[0, 1, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 1, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 1, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 1, 0, 0, '950180'], [1, 0, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 1, 0, 0, '950180'], [0, 0, 0, 1, '950180'], [0, 0, 0, 1, '950180']]
